<a href="https://colab.research.google.com/github/Sarita-jayswal/Assignment-html/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from nltk.tokenize import word_tokenize
import nltk

#from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string


In [2]:
# Define the base path where the data is extracted
data_path = r'C:\Users\DELL\Desktop\torrens-isy\Assign-3\filtered1.csv'

In [3]:
filtered_reviews_df =pd.read_csv(r'/content/filtered1.csv')

In [4]:
filtered_reviews_df.head()


,review_text,rating,title,reviewer,reviewer_location,label,category,cleaned_text,token_count
0,Sphere by Michael Crichton is an excellant nov...,5.0,One of the best Crichton novels,Joseph M,"Colorado, USA",1,books,sphere michael crichton excellant novel certai...,93
1,Dr. Oz is an accomplished heart surgeon in the...,4.0,The Medicine of the Future,Wafa Rashed,"Jabriya, KUWAIT",1,books,accomplished heart surgeon field cardiac trans...,63
2,The most gorgeous artwork in comic books. Cont...,5.0,Beautiful!,"Sarah Silva ""Sar""","San Diego, CA USA",1,books,gorgeous artwork comic books contains extraord...,27
3,This book is for lovers of Robicheaux. His de...,4.0,For lovers of Robicheaux,G. Rousseau,"Finistere, France",1,books,book lovers robicheaux demons seem rest redemp...,42
4,This the one book that taught me more about ho...,4.0,To the point and beautifully Illustrated,RISHAD S. MEHTA,"Bombay, INDIA",1,books,book taught camera manual step step approach l...,73


In [5]:
# Shuffle the entire DataFrame to ensure randomness
filtered_reviews_df = filtered_reviews_df.sample(frac=1, random_state=42).reset_index(drop=True)


Tokenize and Pad the data


In [6]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

#  Randomize the data
filtered_reviews_df = filtered_reviews_df.sample(frac=1, random_state=42).reset_index(drop=True)

#  Tokenization and Padding
max_vocab_size = 20000  # Limit vocabulary size to 20,000 most frequent words
max_seq_len = 120      # Maximum sequence length for padding

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")

# Fit the tokenizer on the cleaned text
tokenizer.fit_on_texts(filtered_reviews_df['cleaned_text'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(filtered_reviews_df['cleaned_text'])

# Replace None tokens with 0
sequences = [[token if token is not None else 0 for token in tokens] for tokens in sequences]

# Pad sequences to ensure uniform input length
padded_sequences = pad_sequences(sequences, maxlen=max_seq_len, padding='post', truncating='post')

# Check the shape of the padded sequences
print(f"Shape of padded sequences: {padded_sequences.shape}")



Shape of padded sequences: (54317, 120)


In [7]:
# Prepare labels for sentiment analysis
sentiment_labels = filtered_reviews_df['label'].values.reshape(-1, 1)  # Assuming 'label' is 1 for positive, 0 for negative
print("Sentiment labels shape:", sentiment_labels.shape)

# One-hot encode the category labels for multi-class classification
category_mapping = {category: idx for idx, category in enumerate(filtered_reviews_df['category'].unique())}
filtered_reviews_df['category_encoded'] = filtered_reviews_df['category'].map(category_mapping).astype(int)
category_labels = filtered_reviews_df['category_encoded'].values  # Ensure shape is (batch_size,)
category_labels = category_labels.astype(int)  # Ensure they are integers
# Flatten the category labels if they have an extra dimension
if len(category_labels.shape) > 1:
    category_labels = category_labels.flatten()

# Print to check
print("Category labels shape:", category_labels.shape)  # Should be (batch_size,)


# Split the data into training, validation, and test sets
X_train, X_temp, y_train_sentiment, y_temp_sentiment, y_train_category, y_temp_category = train_test_split(
    padded_sequences, sentiment_labels, category_labels, test_size=0.3, random_state=42)

X_val, X_test, y_val_sentiment, y_test_sentiment, y_val_category, y_test_category = train_test_split(
    X_temp, y_temp_sentiment, y_temp_category, test_size=0.5, random_state=42)

# Check data shapes
print(f"Training set size: {len(X_train)}, Validation set size: {len(X_val)}, Test set size: {len(X_test)}")
print(f"y_train_category shape after flattening: {y_train_category.shape}")
print(f"y_val_category shape after flattening: {y_val_category.shape}")

Sentiment labels shape: (54317, 1)
Category labels shape: (54317,)
Training set size: 38021, Validation set size: 8148, Test set size: 8148
y_train_category shape after flattening: (38021,)
y_val_category shape after flattening: (8148,)


Model Architecture


In [8]:
print("Shape of y_train_category:", y_train_category.shape)
print("First few entries of y_train_category:", y_train_category[:5])

print("Shape of y_train_sentiment:", y_train_sentiment.shape)
print("First few entries of y_train_sentiment:", y_train_sentiment[:5])

Shape of y_train_category: (38021,)
First few entries of y_train_category: [0 0 0 0 0]
Shape of y_train_sentiment: (38021, 1)
First few entries of y_train_sentiment: [[0]
 [1]
 [1]
 [1]
 [1]]


In [9]:
from tensorflow.keras import backend as K
import gc
# Clear the previous model
K.clear_session()
gc.collect()

0

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout

# Define the model
input_layer = Input(shape=(max_seq_len,))
embedding_layer = Embedding(input_dim=max_vocab_size, output_dim=128, input_length=max_seq_len)(input_layer)
lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)
dropout_layer = Dropout(0.5)(lstm_layer)
lstm_layer_2 = LSTM(128)(dropout_layer)

# Sentiment analysis output branch (binary classification)
sentiment_output = Dense(1, activation='sigmoid', name='sentiment_output')(lstm_layer_2)

# Category classification output branch (multi-class classification)
category_output = Dense(len(category_mapping), activation='softmax', name='category_output')(lstm_layer_2)

# Compile the model using sparse_categorical_crossentropy for the category classification
model = Model(inputs=input_layer, outputs=[sentiment_output, category_output])
model.compile(optimizer='adam',
              loss={'sentiment_output': 'binary_crossentropy', 'category_output': 'sparse_categorical_crossentropy'},
              metrics={'sentiment_output': 'accuracy', 'category_output': 'accuracy'})

# Print model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 120)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 120, 128)       │      2,560,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 120, 128)       │        131,584 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 120, 128)       │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 128)            │        131,584 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sentiment_output (Dense)  │ (None, 1)              │            129 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ category_output (Dense)   │ (None, 4)              │            516 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,823,813 (10.77 MB)

 Trainable params: 2,823,813 (10.77 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
print("X_train shape:", X_train.shape)
print("y_train_sentiment shape:", y_train_sentiment.shape)  # Should be (batch_size, 1)
print("y_train_category shape:", y_train_category.shape)    # Should be (batch_size,)


X_train shape: (38021, 120)
y_train_sentiment shape: (38021, 1)
y_train_category shape: (38021,)


In [12]:
# Train the model
history = model.fit(X_train,
                    {'sentiment_output': y_train_sentiment, 'category_output': y_train_category},
                    validation_data=(X_val, {'sentiment_output': y_val_sentiment, 'category_output': y_val_category}),
                    epochs=10,
                    batch_size=64)


Epoch 1/10
595/595 ━━━━━━━━━━━━━━━━━━━━ 372s 618ms/step - category_output_accuracy: 0.4655 - loss: 1.6770 - sentiment_output_accuracy: 0.7897 - val_category_output_accuracy: 0.4621 - val_loss: 1.6777 - val_sentiment_output_accuracy: 0.7874
Epoch 2/10
595/595 ━━━━━━━━━━━━━━━━━━━━ 363s 587ms/step - category_output_accuracy: 0.4745 - loss: 1.6410 - sentiment_output_accuracy: 0.7932 - val_category_output_accuracy: 0.4699 - val_loss: 1.6382 - val_sentiment_output_accuracy: 0.7874
Epoch 3/10
595/595 ━━━━━━━━━━━━━━━━━━━━ 399s 616ms/step - category_output_accuracy: 0.4734 - loss: 1.5935 - sentiment_output_accuracy: 0.7935 - val_category_output_accuracy: 0.4669 - val_loss: 1.5750 - val_sentiment_output_accuracy: 0.7874
Epoch 4/10
595/595 ━━━━━━━━━━━━━━━━━━━━ 377s 607ms/step - category_output_accuracy: 0.4790 - loss: 1.5383 - sentiment_output_accuracy: 0.7967 - val_category_output_accuracy: 0.4719 - val_loss: 1.5704 - val_sentiment_output_accuracy: 0.7874
Epoch 5/10
595/595 ━━━━━━━━━━━━━━━━━━━━ 